# NeMo ASR Training Using AWS SageMaker

In this tutorial we show how you can train a NeMo ASR Model using [Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/whatis.html). This is meant to be a minimalistic example of how to use SageMaker with NeMo.

AWS SageMaker is useful for practitioners/researchers who are familiar with training locally or on a remote instance (via SSH). SageMaker also supports multi-GPU & Multi-node.

Using AWS SageMaker we train a simple Conformer CTC model using the AN4 dataset on a remote instance with a GPU (p3.2xlarge). We use S3 to store the data and our checkpoints/logs.

The overall steps are:

1. Setup your AWS Credentials to access SageMaker
2. Download the source code we'll be running
3. Setup AN4 dataset, upload data to S3
4. Configure the training job
5. Run training job on SageMaker
6. Download model, (Optional) Tensorboard Logs

In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
BRANCH = 'r1.22.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""
# exit()

In [ ]:
pip install sagemaker awscli

### 1. Setup SageMaker with AWS Credentials

If you haven't setup your AWS credentials, you can setup using the AWS CLI.
You will need your access and Secret key, with permissions to use SageMaker and S3.

In [ ]:
!aws configure

In [ ]:
%load_ext tensorboard
from pathlib import Path
import os
import sagemaker
import wget
from omegaconf import OmegaConf
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

from nemo.utils.notebook_utils import download_an4

In [ ]:
sess = sagemaker.Session()

### 2. Download the NeMo source code

SageMaker allows you to pass in your own source code, with an entrypoint script.

Below we download the AWS NeMo `config.yaml` which contains our configuration, and the `speech_to_text_ctc.py` script to run training.

Our folder structure will look like this:

    code/
        speech_to_text_ctc.py
        conf/
            config.yaml
    data/
        an4/

In [ ]:
root_dir = Path('./an4_nemo_sagemaker/')
code_dir = root_dir / 'code/'
config_dir = code_dir / 'conf/'
data_dir = root_dir / 'data/'

root_dir.mkdir(exist_ok=True)
code_dir.mkdir(exist_ok=True)
config_dir.mkdir(exist_ok=True)
data_dir.mkdir(exist_ok=True)

In [ ]:
config_path = str(config_dir / "config.yaml")
wget.download("https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/asr/conf/conformer/conformer_ctc_char.yaml", config_path)
wget.download("https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/asr/asr_ctc/speech_to_text_ctc.py", str(code_dir))

We also create a `requirements.txt` file within our source code to install NeMo.

In [ ]:
with open(code_dir / 'requirements.txt', 'w') as f:
    f.write("nemo_toolkit[all]")

### 2.1 Initialize SageMaker within Training Script

We provide a helper function that we require to be imported and run at the top of the training script.

This installs and setups DDP for you. It also alleviates having to import a custom container, and can leverage all of the SageMaker containers. Rather than running this cell, you could also manually do this in your script.

In [ ]:
line = "from nemo.utils.cloud import initialize_sagemaker; initialize_sagemaker()"
with open(code_dir / "speech_to_text_ctc.py", 'r+') as f:
    content = f.read()
    f.seek(0, 0)
    f.write(line.rstrip('\r\n') + '\n' + content)

### 3. Setup the AN4 Dataset, upload data to S3

We now download our training and validation data, uploading to S3 so that SageMaker can mount our data to the instance at runtime.

In [ ]:
# within the SageMaker container, mount_dir will be where our data is stored.
download_an4(
    data_dir=str(data_dir),
    train_mount_dir="/opt/ml/input/data/training/",
    test_mount_dir="/opt/ml/input/data/testing/",
)

# Upload to the default bucket
prefix = "an4"
bucket = sess.default_bucket()
loc = sess.upload_data(path=str(data_dir), bucket=bucket, key_prefix=prefix)

In [ ]:
output_path = "s3://" + sess.default_bucket() + "/nemo-output/"

### 4. Configure the training job

Now we configure the training job, by modifying the `config.yaml` file that is stored in our source code directory.
We pass relative directory paths for the data based on the SageMaker mount directory on the remote instance.

In [ ]:
conf = OmegaConf.load(config_path)

# Set Data Locations based on the mounted directory in the SageMaker instance
conf.model.train_ds.manifest_filepath = "/opt/ml/input/data/training/an4/train_manifest.json"
conf.model.validation_ds.manifest_filepath = "/opt/ml/input/data/testing/an4/test_manifest.json"
conf.trainer.accelerator = "gpu"
conf.trainer.max_epochs = 150

# Output directory for our experiment within the SageMaker instance
conf.exp_manager.exp_dir="/opt/ml/model/"

# Create a Small Variant of the Conformer Model
conf.model.encoder.n_layers = 8
conf.model.n_heads = 4
conf.model.spec_augment.time_masks = 5

# Set Optimizer parameters
conf.model.optim.lr = 2.0 # by default we using Noam scheduling, the LR is a multiplier 

OmegaConf.save(conf, config_dir / 'config.yaml')

### 5. Run training on SageMaker

Pass the path of the training and validation data on S3 + the output directory on S3 to the PyTorch estimator, and call fit with the appropriate bucket locations for the training and testing data.

In [ ]:
channels = {"training": loc, "testing": loc}

role = get_execution_role()

# Set to True to enable SageMaker to run locally
local_mode = False

if local_mode:
    instance_type = "local_gpu"
else:
    instance_type = "ml.p3.2xlarge"

est = PyTorch(
    entry_point="speech_to_text_ctc.py", # the script we want to run
    source_dir=str(code_dir), # where our conf/script is
    role=role,
    instance_type=instance_type,
    instance_count=1,
    framework_version="1.12.0", # version of PyTorch
    py_version="py38",
    volume_size=250,
    output_path=output_path,
    hyperparameters={'config-path': 'conf'},
)

est.fit(inputs=channels)

### 6. Download model, (Optional) Tensorboard Logs

SageMaker stores our models/logs within a tar file after training has finished. These can be obtained from S3 like below.

We also visualize the training logs. We suggest using an external logger (such as W&B) to track training progress during the run.

In [ ]:
key = est.model_data.replace("s3://" + sess.default_bucket() + '/', '')

sess.boto_session.client("s3", region_name=sess.boto_region_name).download_file(
    Bucket=sess.default_bucket(), Key=key, Filename='model.tar.gz',
)

In [ ]:
!tar -xvzf model.tar.gz

In [ ]:
%tensorboard --logdir ./ --host 0.0.0.0